In [ ]:
from matplotlib.pyplot import *
from numpy import *
import numpy.random as npr
rcParams['text.usetex'] = True

In [ ]:
from freecad.optics_design_workbench import distributions
from freecad.optics_design_workbench.distributions import *

# Test 1D point generators for given densities (ray fans)

In [ ]:
Xrand = npr.normal(5, 1, size=150)
hX, hDensity = calcHistDensity(Xrand)
dX, dDensity = calcDiffDensity(Xrand)
plot(hX, hDensity, '.')
plot(dX, dDensity, '.')
xlabel('x')
ylabel('density')

### Gaussian

In [ ]:
X = linspace(-1, 2, 500)
Y = exp(-5*X**2)
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=20)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'beliebige Verteilungsfunktion\\als X,Y arrays angegeben')
xlabel('x')
ylabel('normierte Dichte')
plot(Xgen, [0]*len(Xgen), 'o', label=r'platzierte Punkte')
plot(dX, dDens/max(dDens), 'x', label=r'aus den Punktabst\"anden\\bestimmte Dichte')
legend()
#io.savefig('~/Desktop/gaussian-platzierung.jpg')

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 1e-2

### Tophat

In [ ]:
X = linspace(-1, 2, 500)
Y = arctan( 1e5 * ( exp(-5*X**2) - .5 ) )/pi + .5
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=10)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'given density')
xlabel('x')
ylabel('normalized density')
plot(Xgen, [0]*len(Xgen), 'o', label=r'generated points')
plot(dX, dDens/max(dDens), 'x', label=r'\parbox{4cm}{density reconstructed from point-distance}')
legend()

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 1e-5

### tests distorted gaussian

In [ ]:
X = linspace(-1, 3, 500)
Y = arctan( 20*exp(-5*X**2) ) * (1+X)
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=25)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'given density')
xlabel('x')
ylabel('normalized density')
plot(Xgen, [0]*len(Xgen), 'o', label=r'generated points')
plot(dX, dDens/max(dDens), 'x', label=r'\parbox{4cm}{density reconstructed from point-distance}')
legend()

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 1e-2

### Test distribution with multiple maxima

In [ ]:
X = linspace(-1, 5, 500)
Y = exp(-5*X**2) + 0.7*exp(-5*(X-2)**2) 
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=30)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'given density')
xlabel('x')
ylabel('normalized density')
plot(Xgen, [0]*len(Xgen), 'o', label=r'generated points')
plot(dX, dDens/max(dDens), 'x', label=r'\parbox{4cm}{density reconstructed from point-distance}')
legend()

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 3e-2

### same with more points

In [ ]:
X = linspace(-1, 5, 500)
Y = exp(-5*X**2) + 0.7*exp(-5*(X-2)**2)
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=90)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'given density')
xlabel('x')
ylabel('normalized density')
plot(Xgen, [0]*len(Xgen), 'o', label=r'generated points')
plot(dX, dDens/max(dDens), 'x', label=r'\parbox{4cm}{density reconstructed from point-distance}')
legend()

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 1e-2